<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Consulta de Datos de Contratación Pública
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>

## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Abril 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Consulta en datos Base Unificada**|
|**Descripción y alcance**|Script para la consulta de bases y exploración de bases en el DATASANDBOX.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC<td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

## 3. Carga de librerías y ambiente de trabajo

En esta sección se cargan las librerías necesarias para el desarrollo del script y se establecen los parámetros de trabajo.

In [1]:
import pandas as pd
pd.__version__

'2.0.1'

## Instalación de paquetes

Se actualiza pandas para usar la versión 2.0.1
También fastparquet se instala para poder leer los archivos parquet
También se instala pyarrow para poder leer los archivos parquet

```
pip install -U pandas
pip install fastparquet
pip install pyarrow
```

In [2]:
import pandas as pd
import os

In [3]:
DF_SECOP_1=pd.read_parquet('../../../Sample_Data/Processed/SECOP_LIMPIO_2020_2023.parquet')
DF_SECOP_1.shape

(4552721, 33)

In [4]:
DF_SECOP_1.columns

Index(['Fuente', 'ID Contrato', 'ID Proceso', 'ID entidad Plataforma',
       'Entidad', 'NIT Entidad', 'Orden', 'Modalidad', 'Estado',
       'Descripción proceso', 'Objeto Contrato', 'Tipo de contrato',
       'Fecha de Firma', 'UNSPSC', 'Nombre Proveedor', 'Documento Proveedor',
       'Tipo de documento proveedor', 'Valor del contrato',
       'Departamento Entidad', 'Ciudad Entidad', 'Inicio de contrato',
       'Fin de contrato', 'URL', 'Año', 'Mes', 'ID', 'Año-Mes', 'NIT_MDM',
       'Codigo Entidad', 'Modalidad Corta', 'Año Firma', 'Mes Firma',
       'Alerta UNSPSC'],
      dtype='object', name='Unificado')

In [5]:
## Agrupamos los contratos por departamento y sumamos el valor de los contratos

DF_SECOP_1.groupby(['Fuente','Modalidad Corta','Departamento Entidad','Fecha de Firma']).agg({'Valor del contrato':'sum','ID':'count'}).reset_index().to_parquet('Consulta_Indicadores_Transaccionalida.parquet')
DF_SECOP_1.groupby(['Fuente','Modalidad Corta','Departamento Entidad','Año','Mes']).agg({'Valor del contrato':'sum','ID':'count'}).reset_index().to_excel('Consulta_Indicadores_Transaccionalidad_AÑO_MES.xlsx')
DF_SECOP_1.groupby(['Fuente','Modalidad Corta','Año','Mes']).agg({'Valor del contrato':'sum','ID':'count'}).reset_index().to_excel('Consulta_Indicadores_Transaccionalidad_Corta_AÑO_MES.xlsx')


In [6]:
DF_Entidades=DF_SECOP_1.groupby(['Fuente','Departamento Entidad','Entidad','Año','Mes']).agg({'Valor del contrato':'sum','ID':'count'})
DF_Entidades.reset_index(inplace=True)
DF_Entidades=DF_Entidades[DF_Entidades['Año']==2023]
DF_Entidades=DF_Entidades[DF_Entidades['Mes']=='04']

DF_Entidades.sort_values(by=['Valor del contrato'],ascending=False,inplace=True)

In [10]:
DF_SECOP_1=DF_SECOP_1[DF_SECOP_1['ID Contrato']!='Convenio de Cofinanciación para el proyecto L2MB']

In [9]:
DF_S1=DF_SECOP_1[DF_SECOP_1['Año Firma']==2022]
DF_S1=DF_S1[DF_S1['Fuente']=='SECOP I']
DF_S1.sort_values(by=['Valor del contrato'],ascending=False,inplace=True)
DF_S1[['Entidad','ID Contrato','Valor del contrato','URL','Objeto Contrato']].head(100).to_excel('Consulta_Indicadores_Transaccionalidad_SECOP_I.xlsx')

In [15]:
#DF_SECOP_1[DF_SECOP_1['Estado']=='enviado Proveedor']
DF_SECOP_1['Estado'].value_counts()

Estado
Celebrado                                      1711195
En ejecución                                    993408
Liquidado                                       721506
Modificado                                      497127
Terminado                                       295685
Cerrado                                         137549
Activo                                          102023
Terminado sin Liquidar                           75416
Cedido                                           15355
Suspendido                                        2763
Convocado                                          579
Adjudicado                                         107
Terminado Anormalmente después de Convocado          5
enviado Proveedor                                    2
Name: count, dtype: int64

In [11]:
DF_SECOP_1.groupby(['Fuente','Año Firma']).agg({'Valor del contrato':'sum','ID':'count'})


Unificado           Valor del contrato      ID
Fuente   Año Firma                            
SECOP I  2020           59855907053560  919880
         2021           60089782176924  901838
         2022           58104283080780  565432
         2023            6917890709771  121559
         2044                  4654720       1
         2075                     2500       1
         2076                        0       1
         2078                     1500       1
         2079                        0       1
         2086                 45865340       6
         2087                 55000000       1
         2088                  8000000       1
         2090               5524679913       3
         2091                 81325000       1
         2092                 97322560       2
         2093               2677502720       7
         2094             213144927885      13
         2095                263052000       4
         2096             555579610547      17
         2097             443651507209      16
         2098               1030642000       5
         2099             105354523694      18
SECOP II 2020           62528370130723  357240
         2021          109312412147697  561563
         2022           95550195218867  710300
         2023          300736959426389  414809

In [25]:
DF_Entidades

Unificado,Fuente,Departamento Entidad,Entidad,Año,Mes,Valor del contrato,ID
200898,SECOP II,Cundinamarca,FONDO DE PASIVO SOCIAL DE FERROCARRILES NACION...,2023,04,732351335193,83
189948,SECOP II,Bogotá D.C.,UNIDAD NACIONAL DE PROTECCIÓN,2023,04,435402667411,84
178091,SECOP II,Bogotá D.C.,AEROCIVIL,2023,04,352042706961,98
151205,SECOP I,Tolima,TOLIMA ALCALDÍA MUNICIPIO DE ROVIRA,2023,04,224574870925,25
184909,SECOP II,Bogotá D.C.,INSTITUTO DE DESARROLLO URBANO,2023,04,200336631131,224
...,...,...,...,...,...,...,...
112786,SECOP I,Nariño,NARIÑO CONCEJO MUNICIPIO DE NARIÑO NARIÑO,2023,04,0,1
112840,SECOP I,Nariño,NARIÑO CONCEJO MUNICIPIO DE PUERRES,2023,04,0,1
112879,SECOP I,Nariño,NARIÑO CONCEJO MUNICIPIO DE SAN BERNARDO,2023,04,0,1
112930,SECOP I,Nariño,NARIÑO CONCEJO MUNICIPIO DE SANTA BARBARA DE ...,2023,04,0,1


In [6]:
## Leemos desde la tabla desde la 3ra fila
DF_UNSPSC=pd.read_excel('../../../Sample_Data/Auxiliar/clasificador_de_bienes_y_servicios_v14_1.xls',skiprows=3)
DF_UNSPSC.head()

,Código Segmento,Nombre Segmento,Código Familia,Nombre Familia,Código Clase,Nombre Clase,Código Producto,Nombre Producto
0,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101501,Gatos
1,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101502,Perros
2,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101504,Visón
3,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101505,Ratas
4,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101506,Caballos


In [ ]:
### Integramos la información de DF_SECOP con DF_UNSPSC

